In [7]:
import pandas as pd 

In [8]:
df_s2 = pd.read_csv('TestDataS2SR/Test_s2_crop_features_2024_08.csv')
df_s2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282 entries, 0 to 281
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   system:index  282 non-null    object 
 1   B11           8 non-null      float64
 2   B12           8 non-null      float64
 3   B3            8 non-null      float64
 4   B4            8 non-null      float64
 5   B8            8 non-null      float64
 6   EVI           8 non-null      float64
 7   ID            282 non-null    object 
 8   NDVI          8 non-null      float64
 9   month         282 non-null    int64  
 10  year          282 non-null    int64  
 11  .geo          282 non-null    object 
dtypes: float64(7), int64(2), object(3)
memory usage: 26.6+ KB


In [9]:
df_s2.head()

,system:index,B11,B12,B3,B4,B8,EVI,ID,NDVI,month,year,.geo
0,00000000000000000000,NaN,NaN,NaN,NaN,NaN,NaN,ID_UrUGR0,NaN,8,2024,"{""type"":""Polygon"",""coordinates"":[[[-7.56267105..."
1,00000000000000000001,NaN,NaN,NaN,NaN,NaN,NaN,ID_3ZmbBi,NaN,8,2024,"{""type"":""Polygon"",""coordinates"":[[[-7.54128517..."
2,00000000000000000002,NaN,NaN,NaN,NaN,NaN,NaN,ID_tPmH4c,NaN,8,2024,"{""type"":""Polygon"",""coordinates"":[[[-7.22294555..."
3,00000000000000000003,NaN,NaN,NaN,NaN,NaN,NaN,ID_rUfFQH,NaN,8,2024,"{""type"":""Polygon"",""coordinates"":[[[-7.22633000..."
4,00000000000000000004,NaN,NaN,NaN,NaN,NaN,NaN,ID_RrthDZ,NaN,8,2024,"{""type"":""Polygon"",""coordinates"":[[[-7.09798815..."


`Test data interpolation using IDs`

In [10]:
import pandas as pd
import os

# Folder containing your monthly CSVs
data_dir = "TestDataS2SR"
year = 2024

# Collect monthly data
dfs = []
for month in range(1, 13):
    file_path = os.path.join(data_dir, f"Test_s2_crop_features_{year}_{month:02d}.csv")
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        df['month'] = month
        # Ensure missing NDVI/EVI are filled with NaN
        for col in ['NDVI', 'EVI', 'crop', 'B11']:
            if col not in df.columns:
                df[col] = None
        dfs.append(df[['ID', 'NDVI', 'EVI', 'month', 'B11']])

# Combine all months
df_all = pd.concat(dfs, ignore_index=True)

# Pivot to wide format
df_wide = df_all.pivot(index='ID', columns='month', values=['NDVI', 'EVI','B11'])

# Flatten MultiIndex columns
df_wide.columns = [f"{var}_{month:02d}" for var, month in df_wide.columns]

# Optionally, keep the crop label (assume it’s consistent across months)
df_crop = df_all[['ID']].drop_duplicates(subset='ID').set_index('ID')

# Join crop info
df_final = df_crop.join(df_wide).reset_index()

# Final check
# print(df_final.shape)   # Should be (953, 1 + 1 + 24) = 953 rows, 26 columns
df_final.head()


,ID,NDVI_01,NDVI_02,NDVI_03,NDVI_04,NDVI_05,NDVI_06,NDVI_07,NDVI_08,NDVI_09,...,B11_03,B11_04,B11_05,B11_06,B11_07,B11_08,B11_09,B11_10,B11_11,B11_12
0,ID_UrUGR0,NaN,NaN,0.786397,0.580805,NaN,NaN,0.427268,NaN,NaN,...,0.151109,0.231813,NaN,NaN,0.292132,NaN,NaN,NaN,0.116726,0.161250
1,ID_3ZmbBi,NaN,NaN,0.113554,0.643585,NaN,NaN,0.289252,NaN,NaN,...,0.556315,0.163214,NaN,NaN,0.381697,NaN,NaN,NaN,0.173898,0.224403
2,ID_tPmH4c,NaN,NaN,NaN,0.855879,NaN,NaN,NaN,NaN,NaN,...,NaN,0.284457,NaN,NaN,NaN,NaN,NaN,NaN,0.154532,0.189337
3,ID_rUfFQH,NaN,NaN,NaN,0.870789,NaN,NaN,NaN,NaN,NaN,...,NaN,0.251777,NaN,NaN,NaN,NaN,NaN,NaN,0.257732,0.184879
4,ID_RrthDZ,0.5843,0.462308,0.811096,0.620813,NaN,NaN,0.764107,NaN,NaN,...,0.259539,0.275991,NaN,NaN,0.225439,NaN,NaN,NaN,0.212417,0.200784


In [11]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282 entries, 0 to 281
Data columns (total 37 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ID       282 non-null    object 
 1   NDVI_01  166 non-null    float64
 2   NDVI_02  165 non-null    float64
 3   NDVI_03  249 non-null    float64
 4   NDVI_04  244 non-null    float64
 5   NDVI_05  82 non-null     float64
 6   NDVI_06  45 non-null     float64
 7   NDVI_07  199 non-null    float64
 8   NDVI_08  8 non-null      float64
 9   NDVI_09  8 non-null      float64
 10  NDVI_10  60 non-null     float64
 11  NDVI_11  268 non-null    float64
 12  NDVI_12  275 non-null    float64
 13  EVI_01   166 non-null    float64
 14  EVI_02   165 non-null    float64
 15  EVI_03   249 non-null    float64
 16  EVI_04   244 non-null    float64
 17  EVI_05   82 non-null     float64
 18  EVI_06   45 non-null     float64
 19  EVI_07   199 non-null    float64
 20  EVI_08   8 non-null      float64
 21  EVI_09   8 non-n

In [12]:
df_interp = df_final.set_index('ID')

# Interpolate NDVI and EVI columns linearly across the row (i.e., across months)
df_interp = df_interp.interpolate(axis=1, method='linear', limit_direction='both')

# Reset index to restore 'ID' as a column
df_interp = df_interp.reset_index()

In [13]:
df_interp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282 entries, 0 to 281
Data columns (total 37 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ID       282 non-null    object 
 1   NDVI_01  282 non-null    float64
 2   NDVI_02  282 non-null    float64
 3   NDVI_03  282 non-null    float64
 4   NDVI_04  282 non-null    float64
 5   NDVI_05  282 non-null    float64
 6   NDVI_06  282 non-null    float64
 7   NDVI_07  282 non-null    float64
 8   NDVI_08  282 non-null    float64
 9   NDVI_09  282 non-null    float64
 10  NDVI_10  282 non-null    float64
 11  NDVI_11  282 non-null    float64
 12  NDVI_12  282 non-null    float64
 13  EVI_01   282 non-null    float64
 14  EVI_02   282 non-null    float64
 15  EVI_03   282 non-null    float64
 16  EVI_04   282 non-null    float64
 17  EVI_05   282 non-null    float64
 18  EVI_06   282 non-null    float64
 19  EVI_07   282 non-null    float64
 20  EVI_08   282 non-null    float64
 21  EVI_09   282 non

`Train data interpolation`

In [14]:
train = pd.read_csv("TrainDataS2SR/s2_crop_features_2024_01.csv")
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 953 entries, 0 to 952
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   system:index  953 non-null    object 
 1   B11           595 non-null    float64
 2   B12           595 non-null    float64
 3   B3            595 non-null    float64
 4   B4            595 non-null    float64
 5   B8            595 non-null    float64
 6   EVI           595 non-null    float64
 7   ID            953 non-null    object 
 8   NDVI          595 non-null    float64
 9   class         953 non-null    int64  
 10  crop          953 non-null    object 
 11  month         953 non-null    int64  
 12  year          953 non-null    int64  
 13  .geo          953 non-null    object 
dtypes: float64(7), int64(3), object(4)
memory usage: 104.4+ KB


In [15]:
import pandas as pd
import os

# Folder containing your monthly CSVs
data_dir = "TrainDataS2SR"
year = 2024

# Collect monthly data
dfs = []
for month in range(1, 13):
    file_path = os.path.join(data_dir, f"s2_crop_features_{year}_{month:02d}.csv")
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        df['month'] = month
        # Ensure missing NDVI/EVI are filled with NaN
        for col in ['NDVI', 'EVI', 'crop', 'class', 'B11']:
            if col not in df.columns:
                df[col] = None
        dfs.append(df[['ID', 'NDVI', 'EVI', 'month', 'B11']])

# Combine all months
df_all2 = pd.concat(dfs, ignore_index=True)

# Pivot to wide format
df_wide2 = df_all2.pivot(index='ID', columns='month', values=['NDVI', 'EVI', 'B11'])

# Flatten MultiIndex columns
df_wide2.columns = [f"{var}_{month:02d}" for var, month in df_wide2.columns]

# Optionally, keep the crop label (assume it’s consistent across months)
df_crop2 = df_all2[['ID']].drop_duplicates(subset='ID').set_index('ID')

# Join crop info
df_final2 = df_crop2.join(df_wide2).reset_index()

# Final check
# print(df_final.shape)   # Should be (953, 1 + 1 + 24) = 953 rows, 26 columns
df_final2.head()


,ID,NDVI_01,NDVI_02,NDVI_03,NDVI_04,NDVI_05,NDVI_06,NDVI_07,NDVI_08,NDVI_09,...,B11_03,B11_04,B11_05,B11_06,B11_07,B11_08,B11_09,B11_10,B11_11,B11_12
0,ID_29PfPM,0.659164,0.392343,0.475333,0.697105,NaN,0.224526,0.686591,NaN,NaN,...,0.266977,0.264939,NaN,0.401121,0.205020,NaN,NaN,NaN,0.252840,0.143489
1,ID_1xOFkc,0.707107,0.293785,0.653992,NaN,NaN,0.620567,0.355368,NaN,NaN,...,0.269883,NaN,NaN,0.196728,0.204175,NaN,NaN,NaN,0.289522,0.200577
2,ID_vACsoM,0.654873,0.547605,0.685143,0.689890,NaN,0.545003,0.460962,NaN,NaN,...,0.243105,0.285875,NaN,0.207377,0.308408,NaN,NaN,NaN,0.257414,0.254152
3,ID_cr48dU,0.591647,0.479471,0.679535,0.688014,NaN,NaN,0.332317,NaN,NaN,...,0.258598,0.260884,NaN,NaN,0.265895,NaN,NaN,NaN,0.227368,0.234724
4,ID_eIbxNP,0.657750,0.460316,0.394360,0.451362,NaN,NaN,0.583766,NaN,NaN,...,0.367753,0.270005,NaN,NaN,0.201663,NaN,NaN,NaN,0.257221,0.272306


In [16]:
df_final2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 953 entries, 0 to 952
Data columns (total 37 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ID       953 non-null    object 
 1   NDVI_01  595 non-null    float64
 2   NDVI_02  595 non-null    float64
 3   NDVI_03  856 non-null    float64
 4   NDVI_04  777 non-null    float64
 5   NDVI_05  274 non-null    float64
 6   NDVI_06  149 non-null    float64
 7   NDVI_07  642 non-null    float64
 8   NDVI_08  20 non-null     float64
 9   NDVI_09  17 non-null     float64
 10  NDVI_10  225 non-null    float64
 11  NDVI_11  907 non-null    float64
 12  NDVI_12  939 non-null    float64
 13  EVI_01   595 non-null    float64
 14  EVI_02   595 non-null    float64
 15  EVI_03   856 non-null    float64
 16  EVI_04   777 non-null    float64
 17  EVI_05   274 non-null    float64
 18  EVI_06   149 non-null    float64
 19  EVI_07   642 non-null    float64
 20  EVI_08   20 non-null     float64
 21  EVI_09   17 non-

In [17]:
df_interp2 = df_final2.set_index('ID')

# Interpolate NDVI and EVI columns linearly across the row (i.e., across months)
df_interp2 = df_interp2.interpolate(axis=1, method='linear', limit_direction='both')

# Reset index to restore 'ID' as a column
df_interp2 = df_interp2.reset_index()

In [18]:
df_interp2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 953 entries, 0 to 952
Data columns (total 37 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ID       953 non-null    object 
 1   NDVI_01  953 non-null    float64
 2   NDVI_02  953 non-null    float64
 3   NDVI_03  953 non-null    float64
 4   NDVI_04  953 non-null    float64
 5   NDVI_05  953 non-null    float64
 6   NDVI_06  953 non-null    float64
 7   NDVI_07  953 non-null    float64
 8   NDVI_08  953 non-null    float64
 9   NDVI_09  953 non-null    float64
 10  NDVI_10  953 non-null    float64
 11  NDVI_11  953 non-null    float64
 12  NDVI_12  953 non-null    float64
 13  EVI_01   953 non-null    float64
 14  EVI_02   953 non-null    float64
 15  EVI_03   953 non-null    float64
 16  EVI_04   953 non-null    float64
 17  EVI_05   953 non-null    float64
 18  EVI_06   953 non-null    float64
 19  EVI_07   953 non-null    float64
 20  EVI_08   953 non-null    float64
 21  EVI_09   953 non

In [19]:
df_interp2.head()

,ID,NDVI_01,NDVI_02,NDVI_03,NDVI_04,NDVI_05,NDVI_06,NDVI_07,NDVI_08,NDVI_09,...,B11_03,B11_04,B11_05,B11_06,B11_07,B11_08,B11_09,B11_10,B11_11,B11_12
0,ID_29PfPM,0.659164,0.392343,0.475333,0.697105,0.460816,0.224526,0.686591,0.689978,0.693366,...,0.266977,0.264939,0.333030,0.401121,0.205020,0.216975,0.228930,0.240885,0.252840,0.143489
1,ID_1xOFkc,0.707107,0.293785,0.653992,0.642850,0.631709,0.620567,0.355368,0.389054,0.422741,...,0.269883,0.245498,0.221113,0.196728,0.204175,0.225511,0.246848,0.268185,0.289522,0.200577
2,ID_vACsoM,0.654873,0.547605,0.685143,0.689890,0.617447,0.545003,0.460962,0.486873,0.512783,...,0.243105,0.285875,0.246626,0.207377,0.308408,0.295659,0.282911,0.270163,0.257414,0.254152
3,ID_cr48dU,0.591647,0.479471,0.679535,0.688014,0.569448,0.450882,0.332317,0.395490,0.458664,...,0.258598,0.260884,0.262555,0.264225,0.265895,0.256264,0.246632,0.237000,0.227368,0.234724
4,ID_eIbxNP,0.657750,0.460316,0.394360,0.451362,0.495497,0.539632,0.583766,0.577893,0.572020,...,0.367753,0.270005,0.247224,0.224444,0.201663,0.215552,0.229442,0.243331,0.257221,0.272306


In [20]:
df_interp2.to_csv("TrainS2SR.csv", index=False)
df_interp.to_csv("TestS2SR.csv", index=False)

#### `Join elevation data, RVI data and ndvi monthly data on IDs`

In [21]:
elevation_train = pd.read_csv("train_elevation.csv")
elevation_train.drop(columns='Unnamed: 0', inplace=True)
s2sr_train = pd.read_csv("TrainS2SR.csv")
s1_train = pd.read_csv("Train_s1_jan_dec.csv")
s1_train.drop(columns=['system:index','.geo', 'year'], errors='ignore')


,ID,RVI,VH,VV,class,crop,month
0,ID_29PfPM,2.521061,-15.250570,-8.996547,1,Cocoa,1.0
1,ID_1xOFkc,2.536398,-14.548930,-8.446228,1,Cocoa,1.0
2,ID_vACsoM,2.568876,-14.736473,-8.376341,1,Cocoa,1.0
3,ID_cr48dU,2.507548,-15.488482,-9.277652,1,Cocoa,1.0
4,ID_eIbxNP,2.535295,-15.290192,-8.984620,1,Cocoa,1.0
...,...,...,...,...,...,...,...
11431,ID_dGzSAd,2.487829,-13.510779,-8.314608,3,Rubber,12.0
11432,ID_WdCWCr,2.890202,-11.116294,-4.325891,3,Rubber,12.0
11433,ID_M2eYkY,2.557937,-14.487558,-8.182156,3,Rubber,12.0
11434,ID_aHw8nI,2.663925,-14.152866,-7.145208,3,Rubber,12.0


In [22]:
#Transform the long data into wide formart 
s1_train["month"] = s1_train["month"].astype(str).str.zfill(2)

def pivot_variable(var):
    pivot = s1_train.pivot(index="ID", columns="month", values=var)
    pivot.columns = [f"{var}_{m}" for m in pivot.columns]
    return pivot

# Pivot RVI, VV, VH
rvi_wide = pivot_variable("RVI")
vv_wide = pivot_variable("VV")
vh_wide = pivot_variable("VH")

# Merge all into one DataFrame on ID
df_s1_wide = pd.concat([rvi_wide, vv_wide, vh_wide], axis=1)
df_s1_wide = df_s1_wide.reset_index()
df_s1_wide 

,ID,RVI_1.0,RVI_10.0,RVI_11.0,RVI_12.0,RVI_2.0,RVI_3.0,RVI_4.0,RVI_5.0,RVI_6.0,...,VH_11.0,VH_12.0,VH_2.0,VH_3.0,VH_4.0,VH_5.0,VH_6.0,VH_7.0,VH_8.0,VH_9.0
0,ID_04oT3P,2.523300,2.535535,2.498743,2.498541,2.445518,2.500937,2.506776,2.529741,2.537933,...,-15.617841,-15.195914,-14.949791,-15.763383,-14.875921,-15.127219,-15.163856,-15.172360,-14.914063,-15.422450
1,ID_04zn1v,2.682851,2.696425,2.693448,2.679863,2.655231,2.685716,2.670660,2.682900,2.683471,...,-15.334088,-14.968002,-15.177289,-15.288238,-15.699831,-14.877732,-14.584450,-14.852366,-14.449098,-15.234073
2,ID_05RXfR,2.692415,2.589874,2.700382,2.558272,2.381384,2.592859,2.569246,2.566440,2.556142,...,-15.359555,-15.094461,-13.494454,-15.457611,-14.627294,-14.877839,-14.986593,-14.963368,-14.422040,-14.641557
3,ID_06xx2d,2.552050,2.617921,2.591571,2.588547,2.592525,2.524859,2.574750,2.596064,2.597337,...,-16.396055,-16.756147,-16.533504,-16.855996,-16.632424,-16.322646,-15.935120,-16.648757,-16.510626,-16.321109
4,ID_07h9UE,2.566683,2.690927,2.670851,2.551805,2.599044,2.594083,2.542481,2.579031,2.495246,...,-15.040792,-15.260288,-15.244485,-15.943703,-14.710053,-15.696884,-14.560459,-14.468552,-15.339557,-14.302915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,ID_zb9U7L,2.531394,2.638837,2.608204,2.590417,2.563706,2.646097,2.614197,2.578163,2.610238,...,-15.017086,-14.987943,-14.819983,-15.327795,-15.581962,-15.765540,-15.396165,-14.452049,-15.238884,-15.118541
949,ID_zeRXwn,2.527342,2.606041,2.578692,2.712966,2.585403,2.766104,2.760445,2.735386,2.715517,...,-11.960451,-13.443628,-12.662454,-13.442233,-14.772951,-14.265721,-13.506520,-12.775393,-14.132450,-12.952109
950,ID_zjGemX,2.570639,2.570217,2.566364,2.582772,2.479144,2.588281,2.630826,2.593142,2.661343,...,-16.301157,-16.643957,-17.687771,-16.907509,-17.814166,-17.443637,-15.880667,-16.056022,-15.167978,-16.456902
951,ID_zlBrAC,2.486339,2.610121,2.546733,2.547525,2.514330,2.446688,2.493191,2.476267,2.548906,...,-14.928903,-15.698308,-15.330054,-15.092441,-15.194979,-15.163941,-15.454499,-15.426914,-14.840618,-15.348746


In [23]:
final_df = elevation_train.copy()

# Merge with Sentinel-2 features
final_df = final_df.merge(s2sr_train, on='ID', how='left')

# Merge with Sentinel-1 features
final_df = final_df.merge(df_s1_wide, on='ID', how='left')

final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 953 entries, 0 to 952
Data columns (total 79 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           953 non-null    object 
 1   year         953 non-null    int64  
 2   crop         953 non-null    object 
 3   class        953 non-null    int64  
 4   geometry     953 non-null    object 
 5   centroid     953 non-null    object 
 6   height_vals  953 non-null    float64
 7   NDVI_01      953 non-null    float64
 8   NDVI_02      953 non-null    float64
 9   NDVI_03      953 non-null    float64
 10  NDVI_04      953 non-null    float64
 11  NDVI_05      953 non-null    float64
 12  NDVI_06      953 non-null    float64
 13  NDVI_07      953 non-null    float64
 14  NDVI_08      953 non-null    float64
 15  NDVI_09      953 non-null    float64
 16  NDVI_10      953 non-null    float64
 17  NDVI_11      953 non-null    float64
 18  NDVI_12      953 non-null    float64
 19  EVI_01  

In [24]:
final_df.to_csv("DATA_TRAIN.csv", index=False)

`Test data merging`

In [25]:
elevation_test = pd.read_csv("test_elevation.csv")
elevation_test.drop(columns='Unnamed: 0', inplace=True)
s2sr_test = pd.read_csv("TestS2SR.csv")
s1_test = pd.read_csv("Test_s1_jan_dec.csv")
s1_test.drop(columns=['system:index','.geo', 'year'], errors='ignore')

,ID,RVI,VH,VV,month
0,ID_UrUGR0,2.566320,-15.710316,-8.783425,1.0
1,ID_3ZmbBi,2.587864,-16.828711,-9.211338,1.0
2,ID_tPmH4c,2.769679,-12.670523,-5.635031,1.0
3,ID_rUfFQH,2.585326,-14.808502,-8.099906,1.0
4,ID_RrthDZ,2.429516,-13.868693,-8.936721,1.0
...,...,...,...,...,...
3379,ID_uftsbC,2.531778,-15.015298,-8.749289,12.0
3380,ID_EaaSu0,2.728709,-14.921388,-6.979366,12.0
3381,ID_th8V1r,2.548295,-14.861481,-8.565318,12.0
3382,ID_oqPUhU,2.743338,-16.066501,-7.377598,12.0


In [26]:
s1_test["month"] = s1_test["month"].astype(str).str.zfill(2)

def pivot_variable(var):
    pivot = s1_test.pivot(index="ID", columns="month", values=var)
    pivot.columns = [f"{var}_{m}" for m in pivot.columns]
    return pivot

# Pivot RVI, VV, VH
rvi_wide = pivot_variable("RVI")
vv_wide = pivot_variable("VV")
vh_wide = pivot_variable("VH")

# Merge all into one DataFrame on ID
test_s1_wide = pd.concat([rvi_wide, vv_wide, vh_wide], axis=1)
test_s1_wide = test_s1_wide.reset_index()
test_s1_wide 

,ID,RVI_1.0,RVI_10.0,RVI_11.0,RVI_12.0,RVI_2.0,RVI_3.0,RVI_4.0,RVI_5.0,RVI_6.0,...,VH_11.0,VH_12.0,VH_2.0,VH_3.0,VH_4.0,VH_5.0,VH_6.0,VH_7.0,VH_8.0,VH_9.0
0,ID_0MZd85,2.616356,2.741043,2.700998,2.662077,2.567356,2.648677,2.623332,2.678208,2.714872,...,-13.691595,-13.685212,-13.930264,-13.799115,-13.951988,-14.685467,-13.704122,-13.816586,-13.382200,-13.839865
1,ID_0NbEqd,2.665315,2.571597,2.686131,2.634131,2.557081,2.626050,2.650631,2.667769,2.589745,...,-15.649673,-17.636771,-16.779195,-16.384846,-16.728956,-16.957749,-14.886075,-15.695678,-15.972678,-16.290715
2,ID_121DoP,2.431028,2.610571,2.439453,2.584928,2.469501,2.670657,2.661649,2.552878,2.678725,...,-12.391572,-14.833871,-14.316818,-14.120310,-15.113281,-15.676184,-14.390305,-12.574838,-12.731116,-13.165037
3,ID_15h9wu,2.489515,2.526094,2.466620,2.381290,2.466150,2.389942,2.443194,2.438298,2.482478,...,-16.010002,-15.455109,-16.325338,-16.138360,-15.279815,-14.501857,-15.044713,-14.793490,-17.153157,-14.765666
4,ID_22Zm7d,2.695897,2.567857,2.639588,2.698821,2.682912,2.673377,2.785238,2.617878,2.653202,...,-16.429700,-17.384270,-16.857075,-15.693214,-17.363237,-15.566301,-15.389079,-16.231436,-15.602186,-15.542076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,ID_yfKMkV,2.565200,2.555346,2.555431,2.643574,2.687998,2.598895,2.628250,2.683440,2.642327,...,-13.873253,-14.502488,-14.412217,-14.893824,-14.679555,-14.066969,-13.959976,-14.340645,-14.069099,-13.605212
278,ID_z7cOrn,2.499630,2.466393,2.596855,2.542336,2.462525,2.497237,2.546567,2.658578,2.547842,...,-15.351398,-15.808666,-14.367386,-15.580193,-14.884559,-14.822972,-13.875098,-16.181361,-15.052813,-14.595212
279,ID_z8o4ha,2.623575,2.766003,2.652931,2.698722,2.677637,2.598749,2.692698,2.632835,2.702742,...,-14.269329,-15.094498,-14.409931,-15.229927,-15.208721,-14.688692,-14.056333,-14.816541,-15.040890,-14.555316
280,ID_zXWLuI,2.606288,2.655094,2.745283,2.695115,2.591686,2.616365,2.704596,2.679241,2.645619,...,-13.487861,-14.039773,-13.103738,-13.914177,-14.321264,-14.315439,-13.682069,-13.704418,-13.671019,-13.565559


In [27]:
final_test = elevation_test.copy()

# Merge with Sentinel-2 features
final_test = final_test.merge(s2sr_test, on='ID', how='left')

# Merge with Sentinel-1 features
final_test = final_test.merge(test_s1_wide, on='ID', how='left')

final_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282 entries, 0 to 281
Data columns (total 77 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           282 non-null    object 
 1   year         282 non-null    int64  
 2   geometry     282 non-null    object 
 3   centroid     282 non-null    object 
 4   height_vals  282 non-null    float64
 5   NDVI_01      282 non-null    float64
 6   NDVI_02      282 non-null    float64
 7   NDVI_03      282 non-null    float64
 8   NDVI_04      282 non-null    float64
 9   NDVI_05      282 non-null    float64
 10  NDVI_06      282 non-null    float64
 11  NDVI_07      282 non-null    float64
 12  NDVI_08      282 non-null    float64
 13  NDVI_09      282 non-null    float64
 14  NDVI_10      282 non-null    float64
 15  NDVI_11      282 non-null    float64
 16  NDVI_12      282 non-null    float64
 17  EVI_01       282 non-null    float64
 18  EVI_02       282 non-null    float64
 19  EVI_03  

In [28]:
final_test.to_csv("DATA_TEST.csv", index=False)